In [1]:
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
from pathlib import Path
import pandas as pd
from factue.utils.vars import PROJECT_ROOT

base_dir = PROJECT_ROOT / "data/raw/persuasion/trial_data/TRIAL-ANNOTATED"
langs = ["PL", "RU", "BG", "SI"]

records = []

for lang in langs:
    lang_path = base_dir / lang
    raw_docs_path = lang_path / "raw-documents"
    subtask1_path = lang_path / "subtask-1-annotations.txt"
    subtask2_path = lang_path / "subtask-2-annotations.txt"

    # Load all raw documents
    raw_docs = {
        f.name: f.read_text(encoding="utf-8") for f in raw_docs_path.glob("*.txt")
    }

    # Load subtask-2 annotations into a dict
    subtask2_map = {}
    if subtask2_path.exists():
        with subtask2_path.open(encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    fname, start, end, *labels = parts
                    key = (fname, int(start), int(end))
                    subtask2_map[key] = labels

    # Parse subtask-1 annotations and extract spans
    with subtask1_path.open(encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 4:
                continue
            fname, start, end, label_bin = parts
            start, end = int(start), int(end)
            text = raw_docs.get(fname, "")[start:end]
            labels_multi = subtask2_map.get((fname, start, end), [])

            records.append(
                {
                    "filename": fname,
                    "start": start,
                    "end": end,
                    "text_lang": lang,
                    "text": text,
                    "label_bin": label_bin.lower() == "true",
                    "labels_multi": labels_multi,
                }
            )

# Create a DataFrame
df = pd.DataFrame(records)

from pathlib import Path

batch_size = 20
output_base = PROJECT_ROOT / "data/preprocessed/persuasion/train"
output_base.mkdir(parents=True, exist_ok=True)

for lang in df["text_lang"].unique():
    df_lang = df[df["text_lang"] == lang].reset_index(drop=True)
    lang_dir = output_base / f"train-{lang.lower()}"
    lang_dir.mkdir(parents=True, exist_ok=True)

    # Save in batches of 20
    for i in range(0, len(df_lang), batch_size):
        batch_df = df_lang.iloc[i : i + batch_size]
        batch_index = i // batch_size
        batch_filename = f"batch{batch_index:04d}.parquet"
        batch_path = lang_dir / batch_filename
        batch_df.to_parquet(batch_path, index=False)
        print(f"Saved {batch_path}")

Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-pl/batch0000.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-pl/batch0001.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-ru/batch0000.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-bg/batch0000.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-bg/batch0001.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-bg/batch0002.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-bg/batch0003.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/train/train-si/batch0000.parquet


In [3]:
df.value_counts("text_lang")

text_lang
BG    75
PL    27
RU    18
SI     9
Name: count, dtype: int64

In [4]:
from collections import Counter

# Flatten all labels_multi lists into one list
all_labels = df["labels_multi"].explode()

# Drop None or NaN values (in case some entries have empty label lists)
all_labels = all_labels.dropna()

# Count occurrences
label_counts = Counter(all_labels)

# Convert to DataFrame for display or saving
label_counts_df = pd.DataFrame(
    label_counts.items(), columns=["label", "count"]
).sort_values("count", ascending=False)

print(label_counts_df)

                               label  count
2                   Appeal_to_Values     23
13                   Loaded_Language     13
3                              Doubt     11
5                         Repetition     11
1                     Appeal_to_Pity     10
12          Appeal_to_Fear-Prejudice      8
15        Questioning_the_Reputation      8
9              Name_Calling-Labeling      7
0                Conversation_Killer      7
4               Appeal_to_Popularity      6
10           False_Dilemma-No_Choice      3
7                     Appeal_to_Time      3
17         Causal_Oversimplification      3
18                           Slogans      3
19  Consequential_Oversimplification      3
21               Appeal_to_Hypocrisy      3
6          Exaggeration-Minimisation      2
16                         Straw_Man      2
20                      Whataboutism      2
22                       Flag_Waving      2
23                       Red_Herring      2
8    Obfuscation-Vagueness-Confu

In [5]:
import yaml
from langchain.prompts import PromptTemplate



with open("prompts.yaml", "r") as f:
    prompt_defs = yaml.safe_load(f)

prompt_defs


FileNotFoundError: [Errno 2] No such file or directory: 'prompts.yaml'